1-Qubit Bloch Simulator — Guided Tutorial

This notebook demonstrates how to:

Initialize a single-qubit state,

Apply a sequence of unitary gates,

Add Kraus noise channels (T1, T2, depolarizing),

Visualize Bloch-ball trajectories, and

Compare noisy vs. noiseless evolution via radius and fidelity.

Run cells top to bottom.

In [ ]:
# Core setup (inline plotting + optional high DPI)

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 120

# For inline animations (no external writer needed)
from IPython.display import HTML

# Package imports from your project
from qiskit.quantum_info import Statevector, DensityMatrix
from bloch_simulator.bloch_simulator_1_qbit import (
    # fixed gates
    H, Y, T, X, S,
    # main API
    continuous_path, radius, fidelity,
    total_noise_steps, discreet_depolarizing, discreet_T1, discreet_T2,
    animate_trajectory
)

Initialize the input state ρ₀

We take the computational basis state |0⟩ and convert it to a DensityMatrix.

In [ ]:
ket0 = Statevector([1, 0])
rho0 = DensityMatrix(ket0)
rho0

Define a unitary sequence

We select a short gate sequence to generate a non-trivial trajectory on the Bloch sphere.

In [ ]:
seq = [H, Y, T, X, H, S]
steps_per_gate = 100
dt = 0.001  # sampling time step used for plots
n_seq = len(seq)
n_seq

Simulate a noiseless continuous evolution

We split each unitary into small rotations (axis-angle) and sample the Bloch vector along the path.

In [ ]:
pts0 = continuous_path(rho0, seq, steps_per_gate=steps_per_gate)
len(pts0)

Add Kraus noise during the evolution

We define a noise window covering all gates and discretize the T1/T2/Depolarizing channels so that repeated application over the window matches a target noise level.

In [ ]:
t_noise_steps = total_noise_steps(0, n_seq, steps_per_gate=steps_per_gate, n_ops=n_seq)

kraus_dep = discreet_depolarizing(0.5, total_steps=t_noise_steps)
kraus_T1  = discreet_T1(t_total=t_noise_steps*dt, T1=0.4, total_steps=t_noise_steps)
kraus_T2  = discreet_T2(t_total=t_noise_steps*dt, T2=0.2, total_steps=t_noise_steps)
kraus_channels = [kraus_dep, kraus_T1, kraus_T2]

pts = continuous_path(
    rho0, seq,
    steps_per_gate=steps_per_gate,
    kraus_channels=kraus_channels,
    noise_start=0, noise_end=n_seq
)
len(pts)

Diagnostics — Radius and Fidelity

Radius shrinks under noise (point moves inside the Bloch ball).

Fidelity compares noisy vs. noiseless trajectories at each sampled time.

In [ ]:
r0 = radius(pts0, dt)
r  = radius(pts,  dt)
F  = fidelity(pts0, pts, dt)

Inline animation of the trajectories

We display both paths (noiseless and noisy) as separate animations.
Using to_jshtml() renders them inline without needing ffmpeg or pillow.

In [ ]:
anim0 = animate_trajectory(pts0, interval_ms=15)
anim  = animate_trajectory(pts,  interval_ms=15)

HTML(anim0.to_jshtml())
HTML(anim.to_jshtml())